In [1]:
import numpy as np
from copy import deepcopy


def prevedi(hex_str):
    return format(int(hex_str, base=16), '04b')

def translate(code):
    return "".join([prevedi(x) for x in code])

def literal_value(bdata, tot_bin="", pointer=0):
    flag = bdata[pointer]
    code = tot_bin + bdata[pointer+1:pointer+5]
    pointer += 5
    if flag == "0":
        return int(code, base=2), pointer
    else:
        # tot_bin += code
        return literal_value(bdata, tot_bin=code, pointer=pointer)

def my_less(my_list):
    return np.less(*my_list)
def my_greater(my_list):
    return np.greater(*my_list)
def my_equal(my_list):
    return np.equal(*my_list)


OPERATORS = {0: np.sum,
             1: np.prod,
             2: np.min,
             3: np.max,
             4: literal_value,
             5: my_greater,
             6: my_less, 
             7: my_equal}

In [2]:
filename = "./data/day16a.txt"
with open(filename, "r") as ff:
    hstring = ff.read()
bdata = translate(hstring)


packet = dict()
packet["packet_version"] = []
packet["packet_type_id"] = []
packet["value"] = []


def parser(bdata, pointer=0):

    global OPERATORS, packet

    if pointer > len(bdata) - 11:
        print("=========== kraj ===========")

    else:
        packet_version = int(bdata[pointer:pointer+3], base=2)
        packet_type_id = int(bdata[pointer+3:pointer+6], base=2)
        pointer += 6
        packet["packet_version"].append(packet_version)
        packet["packet_type_id"].append(packet_type_id)
        if packet_type_id == 4: # Literal
            nombre, pointer = literal_value(bdata, pointer=pointer)
            packet["value"].append(nombre)
            return nombre, pointer
            # parser(bdata, pointer=pointer)
        else:  # Operator
            subpackets_values = []
            length_type_id = bdata[pointer]
            pointer += 1
            if length_type_id == "0":
                total_length = int(bdata[pointer:pointer+15], base=2)
                pointer += 15
                granica = deepcopy(pointer + total_length)
                while pointer < granica - 5:
                    nombre, pointer = parser(bdata, pointer=pointer)
                    subpackets_values.append(nombre)

            else:
                nb_subpackets = int(bdata[pointer:pointer+11], base=2)
                pointer += 11
                for prd in range(nb_subpackets):
                    nombre, pointer = parser(bdata, pointer=pointer)
                    subpackets_values.append(nombre)
            packet["value"].append(OPERATORS[packet_type_id](subpackets_values))
            return OPERATORS[packet_type_id](subpackets_values), pointer

In [3]:
# Part 2
parser(bdata)[0]

144595909277

In [4]:
# Part 1
np.sum(packet["packet_version"])

993